# Food API

In [6]:
import os
import requests
import time
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def get_ingredients_from_dish(dish_name):
    """
    Fetch ingredients for a dish using Spoonacular API, with a delay between requests.
    """
    # Fetch API key from environment variable
    api_key = os.getenv("api_spoonar")
    if not api_key:
        raise ValueError("API key not found. Please ensure it is set in the .env file.")
    
    # Construct the URL for searching the dish
    url = f"https://api.spoonacular.com/recipes/complexSearch?query={dish_name}&number=1&apiKey={api_key}"
    
    # Wait for 2 seconds to respect API rate limits
    time.sleep(2)
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            # Extract recipe ID from the response
            recipe_id = data["results"][0]["id"]
            
            # Fetch recipe details
            detail_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={api_key}"
            
            # Wait for 2 seconds before making another request
            time.sleep(2)
            detail_response = requests.get(detail_url)
            
            if detail_response.status_code == 200:
                recipe_details = detail_response.json()
                # Extract and return the ingredient names
                return [ingredient["name"] for ingredient in recipe_details["extendedIngredients"]]
    
    # Return an empty list if no results or an error occurred
    return []

# Example usage
if __name__ == "__main__":
    dish_name = input("Enter the name of the dish: ").strip()
    try:
        ingredients = get_ingredients_from_dish(dish_name)
        if ingredients:
            print(f"Ingredients for {dish_name}:")
            for ingredient in ingredients:
                print(f"- {ingredient}")
        else:
            print(f"No ingredients found for {dish_name}.")
    except ValueError as e:
        print(e)


No ingredients found for cheeseburger.


In [11]:
import requests

import os
import requests
import time
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def get_ingredients_from_dish(dish_name):
    """
    Fetch ingredients for a dish using Spoonacular API, with a delay between requests.
    """
    # Fetch API key from environment variable
    api_key = os.getenv("api_spoonar")
    if not api_key:
        raise ValueError("API key not found. Please ensure it is set in the .env file.")
    
    # Construct the URL for searching the dish
    url = f"https://api.spoonacular.com/recipes/complexSearch?query={dish_name}&number=1&apiKey={api_key}"
    
    # Wait for 2 seconds to respect API rate limits
    time.sleep(2)
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        print(f"DEBUG: ComplexSearch API response: {data}")  # Debug API response
        if data.get("results"):
            # Extract recipe ID from the response
            recipe_id = data["results"][0]["id"]
            
            # Fetch recipe details
            detail_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={api_key}"
            
            # Wait for 2 seconds before making another request
            time.sleep(2)
            detail_response = requests.get(detail_url)
            
            if detail_response.status_code == 200:
                recipe_details = detail_response.json()
                print(f"DEBUG: Recipe details response: {recipe_details}")  # Debug Recipe details
                # Extract and return the ingredient names
                return [ingredient["name"] for ingredient in recipe_details["extendedIngredients"]]
        else:
            print(f"No results found for '{dish_name}'. Try another dish name or check spelling.")
    
    return []  # Return an empty list if no results or an error occurred

if __name__ == "__main__":
    dish_name = input("Enter the name of the dish: ").strip()
    try:
        ingredients = get_ingredients_from_dish(dish_name)
        if ingredients:
            print(f"Ingredients for {dish_name}:")
            for ingredient in ingredients:
                print(f"- {ingredient}")
        else:
            print(f"No ingredients found for {dish_name}. Please check the dish name.")
    except ValueError as e:
        print(e)


def fetch_emission_factor(ingredient):
    """
    Fetch emission factor dynamically from a database or API.
    For simplicity, we mock a basic dynamic emission fetcher.
    """
    url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={ingredient}&search_simple=1&json=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        products = data.get("products", [])
        if products:
            # Retrieve category and use it to approximate emissions
            categories = products[0].get("categories", "").lower()
            if "meat" in categories:
                return 50  # Approximation for meats (kg CO2 per kg)
            elif "cheese" in categories:
                return 21  # Approximation for cheese
            elif "vegetables" in categories or "fruit" in categories:
                return 0.5  # Approximation for vegetables
            elif "pasta" in categories or "cereals" in categories:
                return 1.6  # Approximation for pasta/flour
    return 0  # Default emission if not found

def calculate_dish_emissions(dish_name, weights):
    """
    Calculate total emissions for a given dish based on its ingredients.
    """
    ingredients = get_ingredients_from_dish(dish_name)
    total_emissions = 0
    details = []

    for ingredient in ingredients:
        weight = weights.get(ingredient, 0)  # Default weight if not provided
        emission_factor = fetch_emission_factor(ingredient)
        emissions = emission_factor * weight
        total_emissions += emissions
        details.append({
            "ingredient": ingredient,
            "weight (kg)": weight,
            "emission_factor (kg CO2/kg)": emission_factor,
            "emissions (kg CO2)": emissions,
        })
    
    return total_emissions, details

def display_results(dish_name, total_emissions, details):
    """
    Display the CO₂ emissions breakdown for a dish.
    """
    print(f"\nCO₂ Emission Breakdown for {dish_name.title()}:")
    for item in details:
        print(f"- {item['ingredient']}: {item['emissions (kg CO2)']} kg CO₂ "
              f"(weight: {item['weight (kg)']} kg, emission factor: {item['emission_factor (kg CO2/kg)']} kg CO₂/kg)")
    print(f"\nTotal CO₂ emissions for {dish_name.title()}: {total_emissions:.2f} kg CO₂")

if __name__ == "__main__":
    # User-defined dish and weights (in kg)
    dish_name = input("Enter the name of the dish: ").strip()
    print(f"Fetching ingredients for {dish_name}...")

    # Prompt user for weights dynamically
    ingredients = get_ingredients_from_dish(dish_name)
    weights = {}
    for ingredient in ingredients:
        try:
            weight = float(input(f"Enter the weight of {ingredient} in kg (e.g., 0.2 for 200g): "))
            weights[ingredient] = weight
        except ValueError:
            print(f"Invalid weight for {ingredient}. Skipping.")

    # Calculate and display emissions
    if ingredients:
        total_emissions, details = calculate_dish_emissions(dish_name, weights)
        display_results(dish_name, total_emissions, details)
    else:
        print(f"No ingredients found for {dish_name}. Please check the dish name.")


No ingredients found for pasta carbonara. Please check the dish name.
Fetching ingredients for cheeseburger...
No ingredients found for cheeseburger. Please check the dish name.


In [15]:
import os
import requests
import time
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def get_recipe_information(dish_name):
    """
    Fetch recipe information for a dish using Spoonacular API.
    """
    # Fetch API key from environment variable
    api_key = os.getenv("api_spoonar")
    if not api_key:
        raise ValueError("API key not found. Please ensure it is set in the .env file.")
    
    # Step 1: Search for the recipe by dish name
    search_url = f"https://api.spoonacular.com/recipes/complexSearch?query={dish_name}&number=1&apiKey={api_key}"
    time.sleep(1)  # Wait to respect API rate limits
    search_response = requests.get(search_url)
    
    if search_response.status_code != 200:
        print(f"Error: Unable to fetch recipe search results. Status Code: {search_response.status_code}")
        print(f"Message: {search_response.json().get('message', 'No message provided')}")
        return []
    
    search_data = search_response.json()
    if not search_data.get("results"):
        print(f"No results found for '{dish_name}'. Try another dish name or check spelling.")
        return []
    
    # Extract the first recipe ID
    recipe_id = search_data["results"][0]["id"]
    
    # Step 2: Fetch detailed recipe information
    recipe_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={api_key}"
    time.sleep(1)  # Wait to respect API rate limits
    recipe_response = requests.get(recipe_url)
    
    if recipe_response.status_code != 200:
        print(f"Error: Unable to fetch recipe information. Status Code: {recipe_response.status_code}")
        print(f"Message: {recipe_response.json().get('message', 'No message provided')}")
        return []
    
    recipe_data = recipe_response.json()
    print(f"DEBUG: Recipe Information: {recipe_data}")  # Debug output for inspection
    
    # Extract and return ingredients
    return [
        {"name": ingredient["name"], "amount": ingredient["amount"], "unit": ingredient["unit"]}
        for ingredient in recipe_data.get("extendedIngredients", [])
    ]

if __name__ == "__main__":
    dish_name = input("Enter the name of the dish: ").strip()
    try:
        ingredients = get_recipe_information(dish_name)
        if ingredients:
            print(f"Ingredients for {dish_name}:")
            for ingredient in ingredients:
                print(f"- {ingredient['name']}: {ingredient['amount']} {ingredient['unit']}")
        else:
            print(f"No ingredients found for {dish_name}.")
    except ValueError as e:
        print(e)


Error: Unable to fetch recipe search results. Status Code: 401
Message: You are not authorized. Please read https://spoonacular.com/food-api/docs#Authentication
No ingredients found for pasta carbonara.


In [16]:
import os
import requests
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

def get_recipe_ingredients(dish_name):
    """
    Récupère les ingrédients d'un plat en utilisant l'API Spoonacular.
    """
    # Récupérer la clé API depuis les variables d'environnement
    api_key = os.getenv("SPOONACULAR_API_KEY")
    if not api_key:
        raise ValueError("Clé API non trouvée. Veuillez la définir dans le fichier .env.")

    # Étape 1 : Rechercher la recette par nom de plat
    search_url = f"https://api.spoonacular.com/recipes/complexSearch"
    search_params = {
        "query": dish_name,
        "number": 1,  # Limiter à une recette pour des résultats précis
        "apiKey": api_key
    }
    search_response = requests.get(search_url, params=search_params)
    if search_response.status_code != 200:
        print(f"Erreur lors de la recherche de la recette. Statut : {search_response.status_code}")
        return []

    search_results = search_response.json().get("results")
    if not search_results:
        print(f"Aucune recette trouvée pour '{dish_name}'.")
        return []

    # Obtenir l'ID de la première recette trouvée
    recipe_id = search_results[0].get("id")

    # Étape 2 : Récupérer les détails de la recette
    recipe_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
    recipe_params = {
        "apiKey": api_key
    }
    recipe_response = requests.get(recipe_url, params=recipe_params)
    if recipe_response.status_code != 200:
        print(f"Erreur lors de la récupération des détails de la recette. Statut : {recipe_response.status_code}")
        return []

    recipe_info = recipe_response.json()
    ingredients = [
        {
            "name": ingredient["name"],
            "amount": ingredient["amount"],
            "unit": ingredient["unit"]
        }
        for ingredient in recipe_info.get("extendedIngredients", [])
    ]

    return ingredients

if __name__ == "__main__":
    dish_name = input("Entrez le nom du plat : ").strip()
    ingredients = get_recipe_ingredients(dish_name)
    if ingredients:
        print(f"Ingrédients pour '{dish_name}' :")
        for ing in ingredients:
            print(f"- {ing['name']}: {ing['amount']} {ing['unit']}")
    else:
        print(f"Aucun ingrédient trouvé pour '{dish_name}'.")


Ingrédients pour 'pasta carbonara' :
- applewood bacon: 4.0 slice
- cremini mushrooms: 1.0 cup
- eggs: 2.0 large
- kosher salt and pepper: 4.0 servings
- leeks: 2.0 medium
- olive oil: 1.0 tablespoon
- orecchiette: 250.0 grams
- parmesan cheese: 0.5 cup
- parsley: 1.0 tablespoon
- freshly peas: 0.5 cup


In [ ]:
import os
import requests
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

def get_recipe_ingredients(dish_name):
    """
    Récupère les ingrédients d'un plat en utilisant l'API Spoonacular.
    """
    api_key = os.getenv("SPOONACULAR_API_KEY")
    if not api_key:
        raise ValueError("Clé API non trouvée. Veuillez la définir dans le fichier .env.")

    # Étape 1 : Rechercher la recette par nom de plat
    search_url = f"https://api.spoonacular.com/recipes/complexSearch"
    search_params = {
        "query": dish_name,
        "number": 1,
        "apiKey": api_key
    }
    search_response = requests.get(search_url, params=search_params)
    if search_response.status_code != 200:
        print(f"Erreur lors de la recherche de la recette. Statut : {search_response.status_code}")
        return []

    search_results = search_response.json().get("results")
    if not search_results:
        print(f"Aucune recette trouvée pour '{dish_name}'.")
        return []

    # Obtenir l'ID de la première recette trouvée
    recipe_id = search_results[0].get("id")

    # Étape 2 : Récupérer les détails de la recette
    recipe_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
    recipe_params = {
        "apiKey": api_key
    }
    recipe_response = requests.get(recipe_url, params=recipe_params)
    if recipe_response.status_code != 200:
        print(f"Erreur lors de la récupération des détails de la recette. Statut : {recipe_response.status_code}")
        return []

    recipe_info = recipe_response.json()
    ingredients = [
        {
            "name": ingredient["name"],
            "amount": ingredient["amount"],
            "unit": ingredient["unit"]
        }
        for ingredient in recipe_info.get("extendedIngredients", [])
    ]

    return ingredients


def fetch_emission_factor(ingredient_name):
    """
    Récupère les émissions de CO₂ pour un ingrédient en utilisant OpenFoodFacts.
    """
    url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={ingredient_name}&search_simple=1&json=1"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erreur lors de la recherche pour '{ingredient_name}'.")
        return 0  # Retourne 0 si aucune donnée n'est disponible

    data = response.json()
    products = data.get("products", [])
    if not products:
        print(f"Aucun produit trouvé pour '{ingredient_name}'.")
        return 0  # Retourne 0 si aucune donnée n'est disponible

    # Essayez de récupérer les émissions de CO₂ directement, sinon utilisez les catégories
    for product in products:
        # Vérifiez s'il existe une donnée sur les émissions de CO₂
        if "carbon-footprint_100g" in product.get("nutriments", {}):
            return product["nutriments"]["carbon-footprint_100g"] / 1000  # Convertir en kg CO₂ par kg

        # Si aucune donnée directe, utilisez les catégories pour une estimation
        categories = product.get("categories_tags", [])
        if "meat" in categories:
            return 50  # Estimation pour la viande
        elif "cheese" in categories:
            return 21  # Estimation pour le fromage
        elif "vegetables" in categories or "fruit" in categories:
            return 0.5  # Estimation pour les légumes/fruits
        elif "pasta" in categories or "cereals" in categories:
            return 1.6  # Estimation pour les pâtes/céréales

    return 0  # Retourne 0 si aucune donnée n'est disponible


def calculate_dish_emissions(dish_name):
    """
    Calcule les émissions totales de CO₂ pour un plat basé sur ses ingrédients.
    """
    ingredients = get_recipe_ingredients(dish_name)
    if not ingredients:
        return 0, []

    total_emissions = 0
    details = []

    for ingredient in ingredients:
        name = ingredient["name"]
        amount = ingredient["amount"]
        unit = ingredient["unit"]

        # Convertir en kg si nécessaire
        if unit in ["g", "gram"]:
            weight_kg = amount / 1000
        elif unit in ["kg", "kilogram"]:
            weight_kg = amount
        else:
            print(f"Unité '{unit}' non gérée pour {name}.")
            weight_kg = 0

        # Récupérer le facteur d'émission
        emission_factor = fetch_emission_factor(name)
        emissions = emission_factor * weight_kg
        total_emissions += emissions

        details.append({
            "ingredient": name,
            "weight (kg)": weight_kg,
            "emission_factor (kg CO2/kg)": emission_factor,
            "emissions (kg CO2)": emissions,
        })

    return total_emissions, details


def display_results(dish_name, total_emissions, details):
    """
    Affiche le détail des émissions de CO₂ pour un plat.
    """
    print(f"\nDétail des émissions de CO₂ pour '{dish_name}':")
    for item in details:
        print(f"- {item['ingredient']}: {item['emissions (kg CO2)']} kg CO₂ "
              f"(poids : {item['weight (kg)']} kg, facteur d'émission : {item['emission_factor (kg CO2/kg)']} kg CO₂/kg)")
    print(f"\nÉmissions totales de CO₂ pour '{dish_name}': {total_emissions:.2f} kg CO₂")


if __name__ == "__main__":
    dish_name = input("Entrez le nom du plat : ").strip()
    total_emissions, details = calculate_dish_emissions(dish_name)
    if details:
        display_results(dish_name, total_emissions, details)
    else:
        print(f"Aucun ingrédient trouvé pour '{dish_name}'.")


Unité 'slice' non gérée pour applewood bacon.
Unité 'cup' non gérée pour cremini mushrooms.
Unité 'large' non gérée pour eggs.


In [ ]:
import os
import requests
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Table de conversion approximative pour les unités vers kg
UNIT_CONVERSIONS = {
    "slice": 0.03,  # 1 tranche = ~30g
    "cup": 0.24,  # 1 tasse = ~240g pour les liquides
    "tablespoon": 0.015,  # 1 cuillère à soupe = ~15g
    "teaspoon": 0.005,  # 1 cuillère à café = ~5g
    "large": 0.065,  # 1 œuf large = ~65g
    "medium": 0.06,  # 1 œuf medium ou légume moyen = ~60g
    "small": 0.04,  # 1 œuf ou légume petit = ~40g
    "grams": 0.001,  # 1 gramme = 0.001 kg
    "kilograms": 1.0,  # 1 kilogramme = 1 kg
}

def get_recipe_ingredients(dish_name):
    """
    Récupère les ingrédients d'un plat en utilisant l'API Spoonacular.
    """
    api_key = os.getenv("SPOONACULAR_API_KEY")
    if not api_key:
        raise ValueError("Clé API non trouvée. Veuillez la définir dans le fichier .env.")

    # Étape 1 : Rechercher la recette par nom de plat
    search_url = f"https://api.spoonacular.com/recipes/complexSearch"
    search_params = {
        "query": dish_name,
        "number": 1,
        "apiKey": api_key
    }
    search_response = requests.get(search_url, params=search_params)
    if search_response.status_code != 200:
        print(f"Erreur lors de la recherche de la recette. Statut : {search_response.status_code}")
        return []

    search_results = search_response.json().get("results")
    if not search_results:
        print(f"Aucune recette trouvée pour '{dish_name}'.")
        return []

    # Obtenir l'ID de la première recette trouvée
    recipe_id = search_results[0].get("id")

    # Étape 2 : Récupérer les détails de la recette
    recipe_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
    recipe_params = {
        "apiKey": api_key
    }
    recipe_response = requests.get(recipe_url, params=recipe_params)
    if recipe_response.status_code != 200:
        print(f"Erreur lors de la récupération des détails de la recette. Statut : {recipe_response.status_code}")
        return []

    recipe_info = recipe_response.json()
    ingredients = [
        {
            "name": ingredient["name"],
            "amount": ingredient["amount"],
            "unit": ingredient["unit"].lower()  # Mettre les unités en minuscules
        }
        for ingredient in recipe_info.get("extendedIngredients", [])
    ]

    return ingredients


def convert_to_kg(amount, unit):
    """
    Convertit une quantité avec unité en kilogrammes.
    """
    if unit in UNIT_CONVERSIONS:
        return amount * UNIT_CONVERSIONS[unit]
    else:
        print(f"Unité '{unit}' non reconnue, ignorée.")
        return 0


def fetch_emission_factor(ingredient_name):
    """
    Récupère les émissions de CO₂ pour un ingrédient en utilisant OpenFoodFacts.
    """
    url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={ingredient_name}&search_simple=1&json=1"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erreur lors de la recherche pour '{ingredient_name}'.")
        return 0  # Retourne 0 si aucune donnée n'est disponible

    data = response.json()
    products = data.get("products", [])
    if not products:
        print(f"Aucun produit trouvé pour '{ingredient_name}'.")
        return 0  # Retourne 0 si aucune donnée n'est disponible

    # Essayez de récupérer les émissions de CO₂ directement, sinon utilisez les catégories
    for product in products:
        if "carbon-footprint_100g" in product.get("nutriments", {}):
            return product["nutriments"]["carbon-footprint_100g"] / 1000  # Convertir en kg CO₂ par kg

        # Si aucune donnée directe, utilisez les catégories pour une estimation
        categories = product.get("categories_tags", [])
        if "meat" in categories:
            return 50
        elif "cheese" in categories:
            return 21
        elif "vegetables" in categories or "fruit" in categories:
            return 0.5
        elif "pasta" in categories or "cereals" in categories:
            return 1.6

    return 0


def calculate_dish_emissions(dish_name):
    """
    Calcule les émissions totales de CO₂ pour un plat basé sur ses ingrédients.
    """
    ingredients = get_recipe_ingredients(dish_name)
    if not ingredients:
        return 0, []

    total_emissions = 0
    details = []

    for ingredient in ingredients:
        name = ingredient["name"]
        amount = ingredient["amount"]
        unit = ingredient["unit"]

        # Convertir en kg si nécessaire
        weight_kg = convert_to_kg(amount, unit)

        # Récupérer le facteur d'émission
        emission_factor = fetch_emission_factor(name)
        emissions = emission_factor * weight_kg
        total_emissions += emissions

        details.append({
            "ingredient": name,
            "weight (kg)": weight_kg,
            "unit": unit,
            "emission_factor (kg CO2/kg)": emission_factor,
            "emissions (kg CO2)": emissions,
        })

    return total_emissions, details


def display_results(dish_name, total_emissions, details):
    """
    Affiche le détail des émissions de CO₂ pour un plat.
    """
    print(f"\nDétail des émissions de CO₂ pour '{dish_name}':")
    for item in details:
        print(f"- {item['ingredient']}: {item['emissions (kg CO2)']} kg CO₂ "
              f"(poids : {item['weight (kg)']} kg, facteur d'émission : {item['emission_factor (kg CO2/kg)']} kg CO₂/kg)")
    print(f"\nÉmissions totales de CO₂ pour '{dish_name}': {total_emissions:.2f} kg CO₂")


if __name__ == "__main__":
    dish_name = input("Entrez le nom du plat : ").strip()
    total_emissions, details = calculate_dish_emissions(dish_name)
    if details:
        display_results(dish_name, total_emissions, details)
    else:
        print(f"Aucun ingrédient trouvé pour '{dish_name}'.")


Unité 'can' non reconnue, ignorée.


In [1]:
import requests

# Emission factors (kg CO₂ per kg of food)
# These are example values; replace with real values from reliable sources
EMISSION_FACTORS = {
    "beef": 60,  # 60 kg CO2 per kg
    "chicken": 6,  # 6 kg CO2 per kg
    "rice": 2.7,  # 2.7 kg CO2 per kg
    "potato": 0.2,  # 0.2 kg CO2 per kg
    "tomato": 0.1,  # 0.1 kg CO2 per kg
    "cheese": 21,  # 21 kg CO2 per kg
    "bread": 1.6,  # 1.6 kg CO2 per kg
}

def fetch_product_data(ingredient):
    """
    Fetch product data from the Open Food Facts API based on the ingredient name.
    """
    url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={ingredient}&search_simple=1&json=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        products = data.get("products", [])
        if products:
            product = products[0]  # Take the first result
            return {
                "name": product.get("product_name", ingredient),
                "categories": product.get("categories", "Unknown"),
            }
    return {"name": ingredient, "categories": "Unknown"}

def calculate_emissions(plate):
    """
    Calculate total CO₂ emissions for a plate based on its components.
    :param plate: dict of ingredient names and their respective weights in kg
    :return: total emissions and details per ingredient
    """
    total_emissions = 0
    details = []
    
    for ingredient, weight in plate.items():
        # Fetch additional information about the ingredient
        product_data = fetch_product_data(ingredient)
        emission_factor = EMISSION_FACTORS.get(ingredient.lower(), 0)  # Default to 0 if not found
        emissions = emission_factor * weight
        total_emissions += emissions
        details.append({
            "ingredient": ingredient,
            "weight (kg)": weight,
            "emission_factor (kg CO2/kg)": emission_factor,
            "emissions (kg CO2)": emissions,
            "categories": product_data["categories"],
        })
    
    return total_emissions, details

def display_results(total_emissions, details):
    """
    Display the results of the CO₂ calculations.
    """
    print("\nMeal CO₂ Emission Breakdown:")
    for item in details:
        print(f"- {item['ingredient']}: {item['emissions (kg CO2)']} kg CO₂ "
              f"(weight: {item['weight (kg)']} kg, emission factor: {item['emission_factor (kg CO2/kg)']} kg CO₂/kg)")
        print(f"  Categories: {item['categories']}")
    print(f"\nTotal CO₂ emissions for the meal: {total_emissions:.2f} kg CO₂")

def get_user_input():
    """
    Interactively get user input for the meal.
    """
    print("Enter the ingredients of your meal. Type 'done' when finished.")
    plate = {}
    
    while True:
        ingredient = input("Ingredient (or 'done' to finish): ").strip().lower()
        if ingredient == 'done':
            break
        
        try:
            weight = float(input(f"Enter the weight of {ingredient} in kg (e.g., 0.2 for 200g): "))
            plate[ingredient] = weight
        except ValueError:
            print("Invalid weight. Please enter a numeric value.")
    
    return plate

if __name__ == "__main__":
    # Get the user's plate
    user_plate = get_user_input()
    if user_plate:
        # Calculate and display results
        total_emissions, details = calculate_emissions(user_plate)
        display_results(total_emissions, details)
    else:
        print("No ingredients entered. Exiting.")


Enter the ingredients of your meal. Type 'done' when finished.
Invalid weight. Please enter a numeric value.

Meal CO₂ Emission Breakdown:
- pomme: 0.0 kg CO₂ (weight: 0.2 kg, emission factor: 0 kg CO₂/kg)
  Categories: Aliments et boissons à base de végétaux,Aliments d'origine végétale,Céréales et pommes de terre,Pains,Pains croustillants,Pains croustillants complets

Total CO₂ emissions for the meal: 0.00 kg CO₂


## requete avec emission

In [2]:
import requests

def fetch_emission_factor(ingredient):
    """
    Fetch the CO₂ emission factor dynamically from Open Food Facts API based on the ingredient name.
    """
    url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={ingredient}&search_simple=1&json=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        products = data.get("products", [])
        if products:
            product = products[0]  # Take the first result
            nutriments = product.get("nutriments", {})
            # Check if carbon footprint per 100g exists
            if "carbon-footprint_100g" in nutriments:
                return nutriments["carbon-footprint_100g"] * 10  # Convert to kg CO₂ per kg
    # Default to 0 if no data is available
    print(f"No CO₂ data found for '{ingredient}'. Assuming 0.")
    return 0

def calculate_emissions(plate):
    """
    Calculate total CO₂ emissions for a plate based on its components.
    :param plate: dict of ingredient names and their respective weights in kg
    :return: total emissions and details per ingredient
    """
    total_emissions = 0
    details = []
    
    for ingredient, weight in plate.items():
        # Fetch emission factor dynamically
        emission_factor = fetch_emission_factor(ingredient)
        emissions = emission_factor * weight
        total_emissions += emissions
        details.append({
            "ingredient": ingredient,
            "weight (kg)": weight,
            "emission_factor (kg CO2/kg)": emission_factor,
            "emissions (kg CO2)": emissions,
        })
    
    return total_emissions, details

def display_results(total_emissions, details):
    """
    Display the results of the CO₂ calculations.
    """
    print("\nMeal CO₂ Emission Breakdown:")
    for item in details:
        print(f"- {item['ingredient']}: {item['emissions (kg CO2)']} kg CO₂ "
              f"(weight: {item['weight (kg)']} kg, emission factor: {item['emission_factor (kg CO2/kg)']} kg CO₂/kg)")
    print(f"\nTotal CO₂ emissions for the meal: {total_emissions:.2f} kg CO₂")

def get_user_input():
    """
    Interactively get user input for the meal.
    """
    print("Enter the ingredients of your meal. Type 'done' when finished.")
    plate = {}
    
    while True:
        ingredient = input("Ingredient (or 'done' to finish): ").strip().lower()
        if ingredient == 'done':
            break
        
        try:
            weight = float(input(f"Enter the weight of {ingredient} in kg (e.g., 0.2 for 200g): "))
            plate[ingredient] = weight
        except ValueError:
            print("Invalid weight. Please enter a numeric value.")
    
    return plate

if __name__ == "__main__":
    # Get the user's plate
    user_plate = get_user_input()
    if user_plate:
        # Calculate and display results
        total_emissions, details = calculate_emissions(user_plate)
        display_results(total_emissions, details)
    else:
        print("No ingredients entered. Exiting.")


Enter the ingredients of your meal. Type 'done' when finished.
No CO₂ data found for 'pomme'. Assuming 0.
No CO₂ data found for 'chicken'. Assuming 0.

Meal CO₂ Emission Breakdown:
- pomme: 0.0 kg CO₂ (weight: 0.8 kg, emission factor: 0 kg CO₂/kg)
- chicken: 0.0 kg CO₂ (weight: 5.0 kg, emission factor: 0 kg CO₂/kg)

Total CO₂ emissions for the meal: 0.00 kg CO₂


In [7]:
import requests

# Default CO₂ emission factors based on categories (kg CO₂ per kg)
CATEGORY_EMISSION_FACTORS = {
    "meats": 50,  # Generic meat
    "poultries": 40,  # Poultry like chicken
    "fruits": 0.5,  # Generic fruits
    "vegetables": 0.4,  # Generic vegetables
    "cheese": 21,  # Cheese
    "pasta": 1.6,  # Pasta
    "cereals": 1.6,  # Cereals
    "bread": 1.6,  # Bread
    "fish": 6,  # Fish
}

def fetch_emission_factor(ingredient):
    """
    Fetch the CO₂ emission factor dynamically from Open Food Facts API based on the ingredient name.
    If no specific CO₂ data is found, estimate using product categories.
    """
    url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={ingredient}&search_simple=1&json=1"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        products = data.get("products", [])
        if not products:
            print(f"No CO₂ data found for '{ingredient}'. Assuming 0.")
            return 0  # No products found
        
        # Debugging: Print categories of the first product
        categories = products[0].get("categories_tags", [])
        print(f"DEBUG: Categories for '{ingredient}' - {categories}")
        
        # Try to retrieve carbon footprint from the first product
        for product in products:
            nutriments = product.get("nutriments", {})
            if "carbon-footprint_100g" in nutriments:
                return nutriments["carbon-footprint_100g"] * 10  # Convert to kg CO₂ per kg

        # If no direct CO₂ data, use categories as a fallback
        for category in categories:
            category = category.split(":")[-1]  # Extract the actual category name (e.g., "meats")
            if category in CATEGORY_EMISSION_FACTORS:
                print(f"Using fallback emission factor for category '{category}': {CATEGORY_EMISSION_FACTORS[category]} kg CO₂/kg")
                return CATEGORY_EMISSION_FACTORS[category]

    print(f"No CO₂ data found for '{ingredient}'. Assuming 0.")
    return 0  # Default to 0 if no data is available

def calculate_emissions(plate):
    """
    Calculate total CO₂ emissions for a plate based on its components.
    :param plate: dict of ingredient names and their respective weights in kg
    :return: total emissions and details per ingredient
    """
    total_emissions = 0
    details = []
    
    for ingredient, weight in plate.items():
        # Fetch emission factor dynamically
        emission_factor = fetch_emission_factor(ingredient)
        emissions = emission_factor * weight
        total_emissions += emissions
        details.append({
            "ingredient": ingredient,
            "weight (kg)": weight,
            "emission_factor (kg CO2/kg)": emission_factor,
            "emissions (kg CO2)": emissions,
        })
    
    return total_emissions, details

def display_results(total_emissions, details):
    """
    Display the results of the CO₂ calculations.
    """
    print("\nMeal CO₂ Emission Breakdown:")
    for item in details:
        print(f"- {item['ingredient']}: {item['emissions (kg CO2)']} kg CO₂ "
              f"(weight: {item['weight (kg)']} kg, emission factor: {item['emission_factor (kg CO2/kg)']} kg CO₂/kg)")
    print(f"\nTotal CO₂ emissions for the meal: {total_emissions:.2f} kg CO₂")

def get_user_input():
    """
    Interactively get user input for the meal.
    """
    print("Enter the ingredients of your meal. Type 'done' when finished.")
    plate = {}
    
    while True:
        ingredient = input("Ingredient (or 'done' to finish): ").strip().lower()
        if ingredient == 'done':
            break
        
        try:
            weight = float(input(f"Enter the weight of {ingredient} in kg (e.g., 0.2 for 200g): "))
            plate[ingredient] = weight
        except ValueError:
            print("Invalid weight. Please enter a numeric value.")
    
    return plate

if __name__ == "__main__":
    # Get the user's plate
    user_plate = get_user_input()
    if user_plate:
        # Calculate and display results
        total_emissions, details = calculate_emissions(user_plate)
        display_results(total_emissions, details)
    else:
        print("No ingredients entered. Exiting.")


Enter the ingredients of your meal. Type 'done' when finished.
DEBUG: Categories for 'apple' - ['en:plant-based-foods-and-beverages', 'en:plant-based-foods', 'en:fruits-and-vegetables-based-foods', 'en:desserts', 'en:fruits-based-foods', 'en:compotes', 'en:apple-compotes', 'en:light-compotes']
No CO₂ data found for 'apple'. Assuming 0.
DEBUG: Categories for 'chickenn' - []
No CO₂ data found for 'chickenn'. Assuming 0.

Meal CO₂ Emission Breakdown:
- apple: 0.0 kg CO₂ (weight: 3.0 kg, emission factor: 0 kg CO₂/kg)
- chickenn: 0.0 kg CO₂ (weight: 2.0 kg, emission factor: 0 kg CO₂/kg)

Total CO₂ emissions for the meal: 0.00 kg CO₂


In [8]:
import requests

# Default CO₂ emission factors based on categories (kg CO₂ per kg)
CATEGORY_EMISSION_FACTORS = {
    "meats": 50,  # Generic meat
    "poultries": 40,  # Poultry like chicken
    "fruits": 0.5,  # Generic fruits
    "vegetables": 0.4,  # Generic vegetables
    "cheese": 21,  # Cheese
    "pasta": 1.6,  # Pasta
    "cereals": 1.6,  # Cereals
    "bread": 1.6,  # Bread
    "fish": 6,  # Fish
}

def fetch_emission_factor(ingredient):
    """
    Fetch the CO₂ emission factor dynamically from Open Food Facts API based on the ingredient name.
    If no specific CO₂ data is found, estimate using product categories.
    """
    url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={ingredient}&search_simple=1&json=1"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        products = data.get("products", [])
        if not products:
            print(f"No CO₂ data found for '{ingredient}'. Assuming 0.")
            return 0  # No products found
        
        # Debugging: Print categories of the first product
        categories = products[0].get("categories_tags", [])
        print(f"DEBUG: Categories for '{ingredient}' - {categories}")
        
        # Try to retrieve carbon footprint from the first product
        for product in products:
            nutriments = product.get("nutriments", {})
            if "carbon-footprint_100g" in nutriments:
                return nutriments["carbon-footprint_100g"] * 10  # Convert to kg CO₂ per kg

        # If no direct CO₂ data, use categories as a fallback
        for category in categories:
            category = category.split(":")[-1]  # Extract the actual category name (e.g., "meats")
            if category in CATEGORY_EMISSION_FACTORS:
                print(f"Using fallback emission factor for category '{category}': {CATEGORY_EMISSION_FACTORS[category]} kg CO₂/kg")
                return CATEGORY_EMISSION_FACTORS[category]

    print(f"No CO₂ data found for '{ingredient}'. Assuming 0.")
    return 0  # Default to 0 if no data is available

def calculate_emissions(plate):
    """
    Calculate total CO₂ emissions for a plate based on its components.
    :param plate: dict of ingredient names and their respective weights in kg
    :return: total emissions and details per ingredient
    """
    total_emissions = 0
    details = []
    
    for ingredient, weight in plate.items():
        # Fetch emission factor dynamically
        emission_factor = fetch_emission_factor(ingredient)
        emissions = emission_factor * weight
        total_emissions += emissions
        details.append({
            "ingredient": ingredient,
            "weight (kg)": weight,
            "emission_factor (kg CO2/kg)": emission_factor,
            "emissions (kg CO2)": emissions,
        })
    
    return total_emissions, details

def display_results(total_emissions, details):
    """
    Display the results of the CO₂ calculations.
    """
    print("\nMeal CO₂ Emission Breakdown:")
    for item in details:
        print(f"- {item['ingredient']}: {item['emissions (kg CO2)']} kg CO₂ "
              f"(weight: {item['weight (kg)']} kg, emission factor: {item['emission_factor (kg CO2/kg)']} kg CO₂/kg)")
    print(f"\nTotal CO₂ emissions for the meal: {total_emissions:.2f} kg CO₂")

def get_user_input():
    """
    Interactively get user input for the meal.
    """
    print("Enter the ingredients of your meal. Type 'done' when finished.")
    plate = {}
    
    while True:
        ingredient = input("Ingredient (or 'done' to finish): ").strip().lower()
        if ingredient == 'done':
            break
        
        try:
            weight = float(input(f"Enter the weight of {ingredient} in kg (e.g., 0.2 for 200g): "))
            plate[ingredient] = weight
        except ValueError:
            print("Invalid weight. Please enter a numeric value.")
    
    return plate

if __name__ == "__main__":
    # Get the user's plate
    user_plate = get_user_input()
    if user_plate:
        # Calculate and display results
        total_emissions, details = calculate_emissions(user_plate)
        display_results(total_emissions, details)
    else:
        print("No ingredients entered. Exiting.")


Enter the ingredients of your meal. Type 'done' when finished.
DEBUG: Categories for 'pomme' - ['en:plant-based-foods-and-beverages', 'en:plant-based-foods', 'en:cereals-and-potatoes', 'en:breads', 'en:crispbreads', 'fr:pains-croustillants-complets']
No CO₂ data found for 'pomme'. Assuming 0.


KeyboardInterrupt: 

In [9]:
import requests

def fetch_emission_factor(ingredient):
    """
    Récupère le facteur d'émission de CO₂ pour un ingrédient à partir de l'API Open Food Facts.
    """
    url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={ingredient}&search_simple=1&json=1"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        products = data.get("products", [])
        if not products:
            print(f"No CO₂ data found for '{ingredient}'. Assuming 0.")
            return 0  # Aucun produit trouvé
        
        # Recherche des données d'empreinte carbone
        for product in products:
            nutriments = product.get("nutriments", {})
            if "carbon-footprint_100g" in nutriments:
                # Convertir en kg CO₂ par kg
                return nutriments["carbon-footprint_100g"] * 10
        
    print(f"No CO₂ data found for '{ingredient}'. Assuming 0.")
    return 0  # Par défaut, si aucune donnée n'est disponible


def calculate_emissions(plate):
    """
    Calcule les émissions totales de CO₂ pour une liste d'aliments.
    :param plate: Dictionnaire contenant les noms d'ingrédients et leurs poids respectifs (en kg).
    :return: Émissions totales et détails par ingrédient.
    """
    total_emissions = 0
    details = []

    for ingredient, weight in plate.items():
        emission_factor = fetch_emission_factor(ingredient)
        emissions = emission_factor * weight
        total_emissions += emissions

        details.append({
            "ingredient": ingredient,
            "weight (kg)": weight,
            "emission_factor (kg CO2/kg)": emission_factor,
            "emissions (kg CO2)": emissions,
        })

    return total_emissions, details


def display_results(total_emissions, details):
    """
    Affiche les résultats des calculs d'émissions de CO₂.
    """
    print("\nDétail des émissions de CO₂ pour votre repas :")
    for item in details:
        print(f"- {item['ingredient']}: {item['emissions (kg CO2)']} kg CO₂ "
              f"(poids : {item['weight (kg)']} kg, facteur d'émission : {item['emission_factor (kg CO2/kg)']} kg CO₂/kg)")
    print(f"\nÉmissions totales de CO₂ pour le repas : {total_emissions:.2f} kg CO₂")


def get_user_input():
    """
    Permet à l'utilisateur de saisir les aliments et leurs poids.
    """
    print("Entrez les ingrédients de votre repas. Tapez 'done' pour terminer.")
    plate = {}

    while True:
        ingredient = input("Aliment (ou 'done' pour terminer) : ").strip().lower()
        if ingredient == 'done':
            break

        try:
            weight = float(input(f"Entrez le poids de {ingredient} en kg (ex. : 0.2 pour 200g) : "))
            plate[ingredient] = weight
        except ValueError:
            print("Poids invalide. Veuillez entrer une valeur numérique.")

    return plate


if __name__ == "__main__":
    # Saisie des ingrédients par l'utilisateur
    user_plate = get_user_input()
    
    if user_plate:
        # Calcul et affichage des résultats
        total_emissions, details = calculate_emissions(user_plate)
        display_results(total_emissions, details)
    else:
        print("Aucun ingrédient saisi. Fin du programme.")


No ingredients found for poulet.
